In [1]:
import boto3
import json
import time
from dotenv import load_dotenv
from datetime import datetime
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [4]:

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='LATEST'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)


In [ ]:
# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details

shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start reading records
read_kinesis_records(stream_name, shard_iterator)

Received order: {'order_id': '0a25ffaf-6c3a-449f-8186-26404589a99c', 'customer_id': 'cus-8013e39c-fd35-46d7-b848-444a1bce3e40', 'total_weight': 5.280120806305849, 'total_volume': 6.732114742872454, 'total_price': 31.778692370129995, 'order_timestamp': '2024-10-06 21:30:27', 'status': 'RECEIVED', 'lat': 40.486373862839756, 'lon': -3.8634417986933403}
Received order: {'order_id': 'b9107a69-7b47-4c3d-90e3-412e0c172c2f', 'customer_id': 'cus-4fbd8f83-38e3-4849-a1d9-90203fe192a8', 'total_weight': 51.694554967777215, 'total_volume': 128.25475237489894, 'total_price': 333.31860252016804, 'order_timestamp': '2024-10-06 21:30:49', 'status': 'RECEIVED', 'lat': 40.48317967431831, 'lon': -3.6175914087175625}
Received order: {'order_id': '8d193412-5ae8-4d1c-9484-be88f42e0da2', 'customer_id': 'cus-75ec4e4d-efcb-4852-bfaa-c9bd34e00d60', 'total_weight': 27.132723946923647, 'total_volume': 350.18619232668374, 'total_price': 470.1481905620631, 'order_timestamp': '2024-10-06 21:31:14', 'status': 'RECEIVED